En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [1]:
file_path = "farmers-protest-tweets-2021-2-4.json"

# PREGUNTA 1

1. Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. Debe incluir las siguientes funciones:
```python
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
Returns: 
[(datetime.date(1999, 11, 15), "LATAM321"), (datetime.date(1999, 7, 15), "LATAM_CHI"), ...]
```
## PREGUNTA 1 - TIME (q1_time)
Para desarrollar la solución a este problema se utilizará una solución cloud basada en Google Cloud Platform. La función contendrá el proceso de ETL para llevar los datos desde el archivo json local hacia Google Cloud Storage y posteriormente modelar esa data en una tabla de bigquery que permita realizar consultas de manera rápida y eficiente.
### ETL
El proceso de extracción, transformación y carga de los archivos en GCP permite llevar los archivos a una plataforma de performance rápida y efectiva.
#### Credenciales
Este desarrollo se realizará utilizando Google Cloud, por lo que se crea un proyecto en GCP llamado "project-latam-challenge". En este proyecto se crea una service account llamada "sa-etl-latam-challenge" que será utilizada para realizar la carga de datos.
```python
import os

# Ruta a archivo de credenciales JSON de Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../credentials/project-latam-challenge-749ce1a96052.json"
```
#### Carga en Dataframe
Se carga en un dataframe el [archivo](https://drive.google.com/file/d/1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis/view?usp=sharing) json declarado como parte del challenge.
```python
import pandas as pd

# Leer el archivo CSV en un DataFrame
df = pd.read_json(file_path,lines=True)

# Mostrar las primeras filas del DataFrame
df.head()
```
#### Carga de dataframe en GCP
##### Carga de archivo en Google Cloud Storage (GCS)
Usando Google Cloud SDK se crea un bucket llamado 'bucket-project-latam-challenge' en el proyecto 'project-latam-challenge' y se carga el archivo directamente a dicho bucket.
```bash
gcloud auth login
gcloud config set project project-latam-challenge

gsutil mb gs://bucket-project-latam-challenge/

gsutil cp farmers-protest-tweets-2021-2-4.json gs://bucket-project-latam-challenge/
```
##### Configuración de carga
```python
from google.cloud import bigquery

# Parámetros
project_id = 'project-latam-challenge'
dataset_id = 'twitter_data'
table_id = 'farmers_protest_tweets_2021'
```
##### Declaración de schema
Se declara explicitamente la estructura del esquema con la data correspondiente, esto permitirá evitar errores en los tipos de dato al cargar la data en Bigquery.
Por motivos de claridad al momento de leer el markdown, se omite el schema que se puede encontrar en el archivo q1_time.py .
##### Proceso de creación de tabla en bigquery
```python
def load_data_from_gcs_to_bigquery(uri, table_id):
    # Inicializa el cliente de BigQuery
    client = bigquery.Client()

    job_config = bigquery.LoadJobConfig(
        schema=schema,
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
        max_bad_records=0,  # No permitir registros malos antes de fallar
        time_partitioning=bigquery.TimePartitioning(
            type_=bigquery.TimePartitioningType.DAY,
            field="date"  # Campo de partición
        )
    )

    load_job = client.load_table_from_uri(
        uri, table_id, job_config=job_config
    )

    print(f'Starting job {load_job.job_id}')
    load_job.result()
    print(f'Job finished.')

    destination_table = client.get_table(table_id)
    print(f'Loaded {destination_table.num_rows} rows.')

if __name__ == '__main__':
    uri = 'gs://bucket-project-latam-challenge/farmers-protest-tweets-2021-2-4.json'
    table_ref = f"{project_id}.{dataset_id}.{table_id}"

    load_data_from_gcs_to_bigquery(uri, table_ref)
```
##### Análisis
```python
from google.cloud import bigquery
from typing import List, Tuple
import datetime

def get_bigquery_client():
    return bigquery.Client()

def run_bigquery_query(query: str):
    client = get_bigquery_client()
    query_job = client.query(query)
    results = query_job.result()
    return results

def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    query = """
    WITH TEMP_DATA_001 AS 
    (
      SELECT DATE(date) AS fecha,
        id,
        user.username AS username
      FROM `project-latam-challenge.twitter_data.farmers_protest_tweets_2021` 
      WHERE DATE(date) IS NOT NULL
    ), TEMP_DATE_001 AS 
    (
      SELECT fecha,
        COUNT(DISTINCT id) AS tweet_qty
      FROM TEMP_DATA_001
      GROUP BY fecha
    ), TEMP_DATE_002 AS 
    (
      SELECT A.*,
        RANK() OVER(ORDER BY tweet_qty DESC, RAND()) AS ranking_tweet
      FROM TEMP_DATE_001 A
    ), TEMP_USER_001 AS 
    (
      SELECT fecha,
        username,
        COUNT(DISTINCT id) AS tweet_qty
      FROM TEMP_DATA_001
      GROUP BY fecha, username
    ), TEMP_USER_002 AS 
    (
      SELECT A.*,
        RANK() OVER(PARTITION BY fecha ORDER BY tweet_qty DESC, RAND()) AS ranking_user
      FROM TEMP_USER_001 A
    ), TEMP_USER_003 AS 
    (
      SELECT A.*
      FROM TEMP_USER_002 A
      WHERE ranking_user=1
    )
    SELECT A.fecha,
      B.username
    FROM TEMP_DATE_002 A
    LEFT JOIN TEMP_USER_003 B
    ON A.fecha=B.fecha 
    WHERE A.ranking_tweet<=10
    ORDER BY ranking_tweet
    """
    
    results = run_bigquery_query(query)
    return [(row.fecha, row.username) for row in results]

if __name__ == "__main__":
    file_path = '../data/farmers_protest_tweets_2021.json'
    top_10_dates = q1_time(file_path)
    for date, user in top_10_dates:
        print(f"Fecha: {date}, Usuario: {user}")
```

In [17]:
# Importa los módulos necesarios
import os
from q1_time import q1_time

# Establece la ruta del archivo de credenciales
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "d:/valen/Documents/GitHub/latam-challenge/credentials/project-latam-challenge-749ce1a96052.json"

# Ruta al archivo JSON
file_path = 'd:/valen/Documents/GitHub/latam-challenge/src/farmers-protest-tweets-2021-2-4.json'

# Llama a la función q1_time
print("Iniciando proceso q1_time...")
top_10_dates = q1_time(file_path)
print(top_10_dates)
print("Proceso completado.")

# Imprime los resultados
for date, user in top_10_dates:
    print(f"Fecha: {date}, Usuario: {user}")


Iniciando proceso q1_time...
Getting BigQuery client
[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]
Proceso completado.
Fecha: 2021-02-12, Usuario: RanbirS00614606
Fecha: 2021-02-13, Usuario: MaanDee08215437
Fecha: 2021-02-17, Usuario: RaaJVinderkaur
Fecha: 2021-02-16, Usuario: jot__b
Fecha: 2021-02-14, Usuario: rebelpacifist
Fecha: 2021-02-18, Usuario: neetuanjle_nitu
Fecha: 2021-02-15, Usuario: jot__b
Fecha: 2021-02-20, Usuario: MangalJ23056160
Fecha: 2021-02-23, Usuario: Surrypuria
Fecha: 2021-02-19, Usuario: Preetm91
